In [69]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
%matplotlib inline
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)
import keras
import warnings # to filter out warnings

warnings.filterwarnings("ignore")


## Load the data from keras

In [70]:

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data() 

X_train = X_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
X_test = X_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

#X_train= X_train.reshape(X_train.shape[0],X_train.shape[1]*X_train.shape[2])
#X_test = X_test.reshape(X_test.shape[0],X_test.shape[1]*X_test.shape[2])
#y_train = y_train.reshape(y_train.shape[0],1)
#y_test = y_test.reshape(y_test.shape[0],1)

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

1 a) CNN model from scratch: Develop a CNN model with 5 convolutional layers (with kernel size= 3, stride =1, padding = “same”, activation function = “relu”) with following MaxPooling layer (Size= 2) and 3 fully connected layer (including one output layer). After each of the Convolutional layer apply Batch Normalization. In the fully connected layer apply dropout (rate 0.50). Show the learning curve. Report performance evaluation on the test data.

In [71]:
from tensorflow.keras import models,layers
#from tensorflow.keras.models import Sequential 
#from tensorflow.keras.models import Dense 
#from tensorflow.keras import regularizers
tf.random.set_seed(1234)
def create_model():
    # Building the CNN model
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu', input_shape=(28, 28, 1)))
    model.add(layers.BatchNormalization())


    for _ in range(4):
        model.add(layers.Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu'))
        model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))  # Output layer


    # Compile the model
    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    return model


model = create_model()
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32,
                    validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)

# Print the test accuracy
print(f"\nTest Accuracy: {test_acc * 100:.2f}%")



Epoch 1/10
1875/1875 [==============================] - 319s 169ms/step - loss: 1.2642 - accuracy: 0.5476 - val_loss: 0.6456 - val_accuracy: 0.7517
Epoch 2/10
1875/1875 [==============================] - 316s 169ms/step - loss: 0.8358 - accuracy: 0.6702 - val_loss: 0.6191 - val_accuracy: 0.7441
Epoch 3/10
1875/1875 [==============================] - 316s 169ms/step - loss: 0.7370 - accuracy: 0.7032 - val_loss: 0.5558 - val_accuracy: 0.7693
Epoch 4/10
1875/1875 [==============================] - 326s 174ms/step - loss: 0.6367 - accuracy: 0.7369 - val_loss: 0.5222 - val_accuracy: 0.7754
Epoch 5/10
1875/1875 [==============================] - 337s 180ms/step - loss: 0.5637 - accuracy: 0.7606 - val_loss: 0.4622 - val_accuracy: 0.8053
Epoch 6/10
1875/1875 [==============================] - 340s 181ms/step - loss: 0.5266 - accuracy: 0.7790 - val_loss: 0.4743 - val_accuracy: 0.8046
Epoch 7/10
1875/1875 [==============================] - 332s 177ms/step - loss: 0.4894 - accuracy: 0.8094 - val_

In [72]:
model.summary()

Model: "sequential_55"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_342 (Conv2D)         (None, 28, 28, 32)        320       
                                                                 
 batch_normalization_342 (Ba  (None, 28, 28, 32)       128       
 tchNormalization)                                               
                                                                 
 conv2d_343 (Conv2D)         (None, 28, 28, 64)        18496     
                                                                 
 batch_normalization_343 (Ba  (None, 28, 28, 64)       256       
 tchNormalization)                                               
                                                                 
 conv2d_344 (Conv2D)         (None, 28, 28, 64)        36928     
                                                                 
 batch_normalization_344 (Ba  (None, 28, 28, 64)     

1b) Apply 5-Fold Cross Validation on the CNN model developed in a and report the average accuracy with standard deviation.

In [73]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# Wrap Keras model into scikit-learn model
model = KerasClassifier(build_fn=create_model, epochs=1, batch_size=32, verbose=1)

# 5-Fold Cross validation
kfold = KFold(n_splits=5, shuffle=True, random_state=1234)

results = cross_val_score(model, X_train, y_train, cv=kfold)

# Print the cross-validation results
print(f"Baseline Accuracy: {results.mean()*100:.2f}% (+/- {results.std()*100:.2f}%)")

375/375 [==============================] - 15s 39ms/step - loss: 0.8227 - accuracy: 0.6572
Baseline Accuracy: 72.74% (+/- 3.52%)


1 c) Apply grid search on the CNN model to find the optimal set of hyperparameters that produce the max performance on the test data. You must train the model using the training data and evaluate model performance using the test dataset. Use grid search for hyperparameter tuning with the following

In [36]:
from tensorflow.keras.optimizers import Adam, Adagrad
from tensorflow.keras.layers import LeakyReLU
import itertools

# Define the grid search parameters
activation_functions = ['relu', LeakyReLU()] 
optimizers = ['adam', 'adagrad']  
batch_sizes = [16, 32, 64]  
learning_rates = [0.001, 0.0001, 0.00001] 

# Placeholder to store the results
grid_search_results = []

# Loop through the parameter combinations
for params in itertools.product(activation_functions, optimizers, batch_sizes, learning_rates):
    act_func, opt, batch, lr = params
    
    # Creating the model
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), padding='same', input_shape=(28, 28, 1)))
    model.add(layers.BatchNormalization())
    if act_func == 'relu':
        model.add(layers.Activation('relu'))
    else:
        model.add(LeakyReLU())
    
    for _ in range(4):
        model.add(layers.Conv2D(64, (3, 3), padding='same'))
        model.add(layers.BatchNormalization())
        if act_func == 'relu':
            model.add(layers.Activation('relu'))
        else:
            model.add(LeakyReLU())
        
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(10, activation='softmax'))  # Output layer

    if opt == 'adam':
        optimizer = Adam(learning_rate=lr)
    else:
        optimizer = Adagrad(learning_rate=lr)
        
    # Compile the model
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, epochs=1, batch_size=batch, verbose=0)  

    # Evaluate the model
    _, test_acc = model.evaluate(X_test, y_test, verbose=0)
    
    grid_search_results.append((act_func, opt, batch, lr, test_acc))
    print(f"Act: {act_func}, Opt: {opt}, Batch: {batch}, LR: {lr}, Test Acc: {test_acc*100:.2f}%")

# Print the best result
best_params = max(grid_search_results, key=lambda x: x[4])
print(f"\nBest Parameters - Act: {best_params[0]}, Opt: {best_params[1]}, Batch: {best_params[2]}, LR: {best_params[3]}, Test Acc: {best_params[4]*100:.2f}%")


Act: relu, Opt: adam, Batch: 16, LR: 0.001, Test Acc: 87.90%
Act: relu, Opt: adam, Batch: 16, LR: 0.0001, Test Acc: 87.10%
Act: relu, Opt: adam, Batch: 16, LR: 1e-05, Test Acc: 84.54%
Act: relu, Opt: adam, Batch: 32, LR: 0.001, Test Acc: 86.19%
Act: relu, Opt: adam, Batch: 32, LR: 0.0001, Test Acc: 85.66%
Act: relu, Opt: adam, Batch: 32, LR: 1e-05, Test Acc: 81.77%
Act: relu, Opt: adam, Batch: 64, LR: 0.001, Test Acc: 87.04%
Act: relu, Opt: adam, Batch: 64, LR: 0.0001, Test Acc: 86.30%
Act: relu, Opt: adam, Batch: 64, LR: 1e-05, Test Acc: 81.82%
Act: relu, Opt: adagrad, Batch: 16, LR: 0.001, Test Acc: 86.49%
Act: relu, Opt: adagrad, Batch: 16, LR: 0.0001, Test Acc: 79.53%
Act: relu, Opt: adagrad, Batch: 16, LR: 1e-05, Test Acc: 53.61%
Act: relu, Opt: adagrad, Batch: 32, LR: 0.001, Test Acc: 85.84%
Act: relu, Opt: adagrad, Batch: 32, LR: 0.0001, Test Acc: 79.23%
Act: relu, Opt: adagrad, Batch: 32, LR: 1e-05, Test Acc: 56.35%
Act: relu, Opt: adagrad, Batch: 64, LR: 0.001, Test Acc: 85.71

1 d) Data Augmentation: Apply five different image augmentation techniques on the Fashion Mnist train data to augment it and then apply the previously designed (from a) model on it.

In [76]:
#Data Augumentation 

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of the ImageDataGenerator class
datagen = ImageDataGenerator(
    rotation_range=10,       # Rotate images up to 10 degrees
    width_shift_range=0.1,   # Shift images horizontally
    height_shift_range=0.1,  # Shift images vertically
    shear_range=0.1,         # Shear transformation
    zoom_range=0.1           # Zoom in on images
)

# Fit the data generator to your training data
datagen.fit(X_train)

# Rebuilding your model from part (a)
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

for _ in range(4):
    model.add(layers.Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))  # Output layer

# Compiling the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the model using the augmented data generator
# Note that steps_per_epoch should usually be total_samples / batch_size
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    steps_per_epoch=len(X_train) / 32, epochs=10,
                    validation_data=(X_test, y_test))

# Evaluate and print test accuracy
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy after Data Augmentation: {test_acc * 100:.2f}%")


Epoch 1/10
1875/1875 [==============================] - 128s 67ms/step - loss: 1.1889 - accuracy: 0.5749 - val_loss: 0.6493 - val_accuracy: 0.7516
Epoch 2/10
1875/1875 [==============================] - 125s 67ms/step - loss: 0.7731 - accuracy: 0.6980 - val_loss: 0.5727 - val_accuracy: 0.7626
Epoch 3/10
1875/1875 [==============================] - 125s 67ms/step - loss: 0.6961 - accuracy: 0.7203 - val_loss: 0.5972 - val_accuracy: 0.7404
Epoch 4/10
1875/1875 [==============================] - 131s 70ms/step - loss: 0.6321 - accuracy: 0.7394 - val_loss: 0.5239 - val_accuracy: 0.7836
Epoch 5/10
1875/1875 [==============================] - 126s 67ms/step - loss: 0.5903 - accuracy: 0.7659 - val_loss: 0.4636 - val_accuracy: 0.8416
Epoch 6/10
1875/1875 [==============================] - 125s 67ms/step - loss: 0.5540 - accuracy: 0.7856 - val_loss: 0.4406 - val_accuracy: 0.8499
Epoch 7/10
1875/1875 [==============================] - 128s 68ms/step - loss: 0.5227 - accuracy: 0.8015 - val_loss: 0

In [78]:
model.summary()

Model: "sequential_61"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_372 (Conv2D)         (None, 28, 28, 32)        320       
                                                                 
 batch_normalization_372 (Ba  (None, 28, 28, 32)       128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_87 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_373 (Conv2D)         (None, 14, 14, 64)        18496     
                                                                 
 batch_normalization_373 (Ba  (None, 14, 14, 64)       256       
 tchNormalization)                                               
                                                     

1 e) Transfer Learning: Load the VGG-19 model. Drop after the block4 conv1 layer (highlighted in the image below) and on top of it add one global average pooling layer, one fully connected layer, and one final output layer. Keep the base model layers (VGG19) freeze

In [55]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

# Load the VGG19 model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Truncate the model at 'block4_conv1'
base_output = base_model.get_layer('block4_conv1').output

# Add new layers on top
x = GlobalAveragePooling2D()(base_output)  # Global Average Pooling layer
x = Dense(128, activation='relu')(x)  # Fully connected layer with 128 neurons
x = Dropout(0.5)(x)  # Dropout layer for regularization
predictions = Dense(10, activation='softmax')(x)  # Output layer assuming 10 classes

# Assemble the final model
final_model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Print the model summary to check the architecture
final_model.summary()


Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

2) Dveloping ResNet model from scratch
Apply a residual network specified in the following architecture. All convolutional layers use kernel size 3, stride = 1, and padding = “same”,

In [89]:
import os
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, GlobalAveragePooling2D, Dense, Flatten
from tensorflow.keras.models import Model, load_model

# Function to create a residual block
def residual_block(x, filters):
    shortcut = x
    
    # Main path
    x = Conv2D(filters, (3,3), strides=(1,1), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(filters, (3,3), strides=(1,1), padding='same')(x)
    x = BatchNormalization()(x)
    
    # If the number of filters does not match, apply a 1x1 conv to the shortcut path
    if shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, (1,1), strides=(1,1), padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)  # Optional: add BatchNormalization here as well
    
    # Adding the shortcut to the output
    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x

# Function to create the ResNet model
def create_resnet_model():
    inputs = Input(shape=(28, 28, 1))  # Adjust the input shape according to your data
    x = Conv2D(32, (3,3), strides=(1,1), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Section A
    for _ in range(3):
        x = residual_block(x, 32)
    
    # Section B
    for _ in range(3):
        x = residual_block(x, 64)
    
    # Section C
    for _ in range(3):
        x = residual_block(x, 128)
    
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)  # Adjust the size and activation function according to your needs
    outputs = Dense(10, activation='softmax')(x)  # Assuming 10 classes for classification

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

model = create_resnet_model()
# Print model summary
model.summary()


Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_16 (InputLayer)          [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d_440 (Conv2D)            (None, 28, 28, 32)   320         ['input_16[0][0]']               
                                                                                                  
 batch_normalization_440 (Batch  (None, 28, 28, 32)  128         ['conv2d_440[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 re_lu_119 (ReLU)               (None, 28, 28, 32)   0           ['batch_normalization_440[

In [50]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d_296 (Conv2D)            (None, 28, 28, 32)   320         ['input_9[0][0]']                
                                                                                                  
 batch_normalization_296 (Batch  (None, 28, 28, 32)  128         ['conv2d_296[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 re_lu_43 (ReLU)                (None, 28, 28, 32)   0           ['batch_normalization_296[0

In [92]:
# Train the model
model.fit(X_train, y_train, epochs=2, batch_size=32,
                    validation_data=(X_test, y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)

# Print the test accuracy
print(f"\nTest Accuracy: {test_acc * 100:.2f}%")

Epoch 1/2
1875/1875 [==============================] - 1818s 967ms/step - loss: 0.4970 - accuracy: 0.8194 - val_loss: 0.6456 - val_accuracy: 0.7833
Epoch 2/2
313/313 [==============================] - 73s 233ms/step - loss: 0.4080 - accuracy: 0.8667

Test Accuracy: 86.67%


In [87]:

model_file = "/Users/nyzy/Library/CloudStorage/OneDrive-Personal/deep_learning/resnet_model.h5"
# Check if the model file exists. If it does, load it. If not, create, train, and save the model.
if os.path.exists(model_file):
    print("Loading existing model")
    model = load_model(model_file)
else:
    print("Creating and training a new model")
    
    # Train the model with your data
    # model.fit(x_train, y_train, epochs=..., batch_size=...)
    history.save(model_file)

Creating and training a new model


In [93]:
model_file = "/Users/nyzy/Library/CloudStorage/OneDrive-Personal/deep_learning/resnet_model.h5"
model.save(model_file)